In [ ]:
import os
import cv2 
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder


In [ ]:
detection_model = tf.saved_model.load('saved_model')

In [ ]:
print(detection_model.signatures['serving_default'].inputs)

In [ ]:
@tf.function
def detect_fn(image):
    model = detection_model.signatures['serving_default']
    detections = model(image)
    return detections

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap('label_map.pbtxt')

In [ ]:
def predict(img, output_directory):  #mandatory function: First arg: img path to read from, Second arg: Dir to save the output image
        img = cv2.imread(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        image_np = np.array(img)
        label_id_offset = 0
        input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0))
        detections = detect_fn(input_tensor)
   
        num_detections = int(detections.pop('num_detections'))
        detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
        detections['num_detections'] = num_detections

        detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
        image_np_with_detections = image_np.copy()
   
        counter = viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],
        (detections['detection_classes']+label_id_offset).astype(int),
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        #max_boxes_to_draw=1,
        min_score_thresh=.5,
        line_thickness=8,
            
        agnostic_mode=False
        )
        output_file = output_directory + "image.jpg"
        cv2.imwrite(output_file,image_np_with_detections)
        return output_file

In [ ]:
predict("2.jpg","")

In [ ]:
#mandatory function: First arg: img path to read from, Second arg: Dir to save the output image
img = "data/test/Cars411.png"
img = cv2.imread(img)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
image_np = np.array(img)
label_id_offset = 0
input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0))
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
detections['num_detections'] = num_detections

detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
image_np_with_detections = image_np.copy()

counter = viz_utils.visualize_boxes_and_labels_on_image_array(
image_np_with_detections,
detections['detection_boxes'],
(detections['detection_classes']+label_id_offset).astype(int),
detections['detection_scores'],
category_index,
use_normalized_coordinates=True,
#max_boxes_to_draw=1,
min_score_thresh=.5,
line_thickness=8,

agnostic_mode=False
)
plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.show()

# Learning how to crop the bounding boxes and use ocr
### To do
#### change ocr for better prediction

In [ ]:
detections

In [ ]:
detections.keys()

In [ ]:
detections['detection_scores']

In [ ]:
min_threshold = 0.5
scores=[]
for i in detections['detection_scores']:
    if i > min_threshold:
        scores.append(i)
scores

In [ ]:
N_scores = len(scores)
boxes = []
for i in range(N_scores):
    boxes.append(detections['detection_boxes'][i])
boxes = np.array(boxes)
boxes.flatten()
boxes

In [ ]:
classes = detections['detection_classes'][:len(scores)]

In [ ]:
'''
score = list(filter(lambda x: x> min_threshold, detections['detection_scores']))
'''
'''
boxes = detections['detection_boxes'][:len(scores)]
'''
image_np_with_detections

In [ ]:

image = image_np_with_detections

width, height = image.shape[1],image.shape[0]

In [ ]:
(height,width)

In [ ]:
boxes

In [ ]:
!pip install easyocr

In [ ]:
import easyocr
for idx, box in enumerate(boxes):
    print(box)
    roi = box*[height, width, height, width]
    print(roi)
    region = image[int(roi[0]):int(roi[2]),int(roi[1]):int(roi[3])]
    print(region.shape[0],region.shape[1])
    reader = easyocr.Reader(['en'])
    ocr_result = reader.readtext(region)
    print(ocr_result)

    plt.imshow(cv2.cvtColor(region, cv2.COLOR_BGR2RGB))
#The output shows 4 bounding box coordinates(x,y) of the text along with the identified text and confidence score.

In [ ]:
for result in ocr_result:
    print(result[0][2],result[0][1])
    print(result[0][1], result[0][0])
    print(np.sum(np.subtract(result[0][2],result[0][1])))
    print(np.sum(np.subtract(result[0][1], result[0][0])))
    print(result[1])

In [ ]:
region_threshold = 0.05

# Predict Function (changes yet to be made)

In [ ]:
import os
import cv2 
import easyocr
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

detection_model = tf.saved_model.load('saved_model')

@tf.function
def detect_fn(image):
    model = detection_model.signatures['serving_default']
    detections = model(image)
    return detections

category_index = label_map_util.create_category_index_from_labelmap('label_map.pbtxt')

def filter_text(region, ocr_result, region_threshold):
    rectangle_size = region.shape[0]*region.shape[1]
    
    plate = [] 
    for result in ocr_result:
        length = np.sum(np.subtract(result[0][1], result[0][0]))
        height = np.sum(np.subtract(result[0][2], result[0][1]))
        
        if length*height / rectangle_size > region_threshold:
            plate.append(result[1])
    return plate

def ocr_it(image, detections, detection_threshold, region_threshold):
    
    # Scores, boxes and classes above threhold
    scores = list(filter(lambda x: x> detection_threshold, detections['detection_scores']))
    boxes = detections['detection_boxes'][:len(scores)]
    classes = detections['detection_classes'][:len(scores)]
    
    # Full image dimensions
    width = image.shape[1]
    height = image.shape[0]
    
    # Apply ROI filtering and OCR
    for idx, box in enumerate(boxes):
        roi = box*[height, width, height, width]
        region = image[int(roi[0]):int(roi[2]),int(roi[1]):int(roi[3])]
        reader = easyocr.Reader(['en'])
        ocr_result = reader.readtext(region)
        
        text = filter_text(region, ocr_result, region_threshold)
        
        plt.imshow(cv2.cvtColor(region, cv2.COLOR_BGR2RGB))
        plt.show()
#         print(text)
        return text, region

# def process_image(iamge_name, lang_code):
#     return pytesseract.image_to_string(Image.open(iamge_name), lang=lang_code)

def predict(img, output_directory):  #mandatory function: First arg: img path to read from, Second arg: Dir to save the output image
        img = cv2.imread(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        image_np = np.array(img)
        label_id_offset = 0
        input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0))
        detections = detect_fn(input_tensor)
   
        num_detections = int(detections.pop('num_detections'))
        detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
        detections['num_detections'] = num_detections

        detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
        image_np_with_detections = image_np.copy()
   
        counter = viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],
        (detections['detection_classes']+label_id_offset).astype(int),
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        #max_boxes_to_draw=1,
        min_score_thresh=.5,
        line_thickness=8,
            
        agnostic_mode=False
        )
        img_rgb = cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB)
        pytesseract.pytesseract.tesseract_cmd = "C:\\Users\\shailesh gadde\\AppData\\Local\\Programs\\Python\\Python37\\Lib\\site-packages\\"
        print(pytesseract.image_to_string(img_rgb))
#         text, region = ocr_it(image_np_with_detections, detections, 0.5, 0.6)
#         print(text)
#         data_eng = process_image("test_eng.png", "eng")
        output_file = output_directory + "image.jpg"
        cv2.imwrite(output_file,image_np_with_detections)
        return output_file


In [ ]:

    
predict('2.jpg',"")

In [225]:
from PIL import Image
import pytesseract
import os
import cv2 
import easyocr
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

detection_model = tf.saved_model.load('saved_model')

@tf.function
def detect_fn(image):
    model = detection_model.signatures['serving_default']
    detections = model(image)
    return detections

[WinError 123] The filename, directory name, or volume label syntax is incorrect: "'C:\\Program Files\\Tesseract-OCR\\tesseract'"
D:\ocr_plate


In [157]:
import easyocr

category_index = label_map_util.create_category_index_from_labelmap('label_map.pbtxt')
img = 'data/test/Cars411.png'
img = cv2.imread(img)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
image_np = np.array(img)
label_id_offset = 0
input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0))
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
detections['num_detections'] = num_detections

detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
image_np_with_detections = image_np.copy()

counter = viz_utils.visualize_boxes_and_labels_on_image_array(
image_np_with_detections,
detections['detection_boxes'],
(detections['detection_classes']+label_id_offset).astype(int),
detections['detection_scores'],
category_index,
use_normalized_coordinates=True,
#max_boxes_to_draw=1,
min_score_thresh=.5,
line_thickness=8,

agnostic_mode=False
)

# pytessercat
pytesseract.pytesseract.tesseract_cmd = "C:\\Program Files\\Tesseract-OCR\\tesseract"


text = pytesseract.image_to_string(image_np_with_detections,lang='eng')
output_file = "image.jpg"
cv2.imwrite(output_file,image_np_with_detections)

True

In [158]:
detections

{'detection_classes': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64),
 'detection_multiclass_scores': array([[0.00565681, 0.71099305],
        [0.00319898, 0.0755901 ],
        [0.0036087 , 0.05917037],
        [0.00313589, 0.04472983],
        [0.00270021, 0.04422605],
        [0.00337189, 0.04224795],
        [0.0050599 , 0.04076853],
        [0.00395188, 0.0319854 ],
        [0.00981438, 0.02980587],
        [0.00117484, 0.02966887],
        [0.00449544, 0.02870139],
        [0.00119135, 0.02848524],
        [0.0023163 , 0.02757576],
        [0.00291127, 0.02730319],
        [0.00201702, 0.02712533],
        [0.00263873, 0.02650225],
        [0.0039897 , 0.02570212],
        [0.0023

In [159]:
min_threshold = 0.5

scores = list(filter(lambda x: x> min_threshold, detections['detection_scores']))

boxes = detections['detection_boxes'][:len(scores)]

classes = detections['detection_classes'][:len(scores)]

In [160]:
print(scores,
boxes,
classes)

[0.71099305] [[0.5649924  0.38808304 0.74030554 0.6308623 ]] [1]


In [161]:
image = image_np_with_detections
os=[]
width, height = image.shape[1],image.shape[0]
for idx, box in enumerate(boxes):
    print(box)
    roi = box*[height, width, height, width]
    print(roi)
    region = image[int(roi[0]):int(roi[2]),int(roi[1]):int(roi[3])]
    print(region.shape[0],region.shape[1])
    ocr_result = pytesseract.image_to_string(region,lang='eng')

    
    plt.imshow(cv2.cvtColor(region, cv2.COLOR_BGR2RGB))

[0.5649924  0.38808304 0.74030554 0.6308623 ]
[151.41797066 155.23321629 198.40188551 252.34491825]
47 97


In [227]:


category_index = label_map_util.create_category_index_from_labelmap('label_map.pbtxt')
img = 'test/num_plate.jpg'
img = cv2.imread(img)
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
image_np = np.array(img)
label_id_offset = 0
input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0))
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
detections['num_detections'] = num_detections

detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
image_np_with_detections = image_np.copy()

counter = viz_utils.visualize_boxes_and_labels_on_image_array(
image_np_with_detections,
detections['detection_boxes'],
(detections['detection_classes']+label_id_offset).astype(int),
detections['detection_scores'],
category_index,
use_normalized_coordinates=True,
#max_boxes_to_draw=1,
min_score_thresh=.3,
line_thickness=8,

agnostic_mode=False
)

# pytessercat
pytesseract.pytesseract.tesseract_cmd = "C:\\Program Files\\Tesseract-OCR\\tesseract"
text, region = ocr_it(image_np_with_detections, detections, 0.3, 0.6)
# config = ('-l eng --oem 1 --psm 3')

# # Run tesseract OCR on image
# text = pytesseract.image_to_string(image_np_with_detections, config=config)
# print(text)
# text = pytesseract.image_to_string(image_np_with_detections,lang='eng')
output_file = "image.jpg"
cv2.imwrite(output_file,image_np_with_detections)

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


[]


c:\users\shailesh gadde\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:35: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


True

In [213]:
# min_threshold = 0.3

# scores = list(filter(lambda x: x> min_threshold, detections['detection_scores']))

# boxes = detections['detection_boxes'][:len(scores)]

# classes = detections['detection_classes'][:len(scores)]

In [228]:
# image = image_np_with_detections
# width, height = image.shape[1],image.shape[0]
# for idx, box in enumerate(boxes):
#     print(box)
#     roi = box*[height, width, height, width]
#     print(roi)
#     region = image[int(roi[0]):int(roi[2]),int(roi[1]):int(roi[3])]
#     print(region.shape[0],region.shape[1])

In [229]:

def filter_text(region, ocr_result, region_threshold):
    rectangle_size = region.shape[0]*region.shape[1]
    
    plate = [] 
    for result in ocr_result:
        length = np.sum(np.subtract(result[0][1], result[0][0]))
        height = np.sum(np.subtract(result[0][2], result[0][1]))
        
        if length*height / rectangle_size > region_threshold:
            plate.append(result[1])
    return plate


def ocr_it(image, detections, detection_threshold, region_threshold):
    
    # Scores, boxes and classes above threhold
    scores = list(filter(lambda x: x> detection_threshold, detections['detection_scores']))
    boxes = detections['detection_boxes'][:len(scores)]
    classes = detections['detection_classes'][:len(scores)]
    
    # Full image dimensions
    width = image.shape[1]
    height = image.shape[0]
    
    # Apply ROI filtering and OCR
    for idx, box in enumerate(boxes):
        roi = box*[height, width, height, width]
        region = image[int(roi[0]):int(roi[2]),int(roi[1]):int(roi[3])]
        reader = easyocr.Reader(['en'])
        ocr_result = reader.readtext(region)
        
        text = filter_text(region, ocr_result, region_threshold)
        
        plt.imshow(cv2.cvtColor(region, cv2.COLOR_BGR2RGB))
        plt.show()
        return text, region

In [223]:
import numpy as np
import cv2
import  imutils
import sys
import pytesseract
import pandas as pd
import time

image = region

# image = imutils.resize(image, width=500)

cv2.imwrite("Original_Image.jpg", image)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#cv2.imshow("1 - Grayscale Conversion", gray)

gray = cv2.bilateralFilter(gray, 11, 17, 17)
#cv2.imshow("2 - Bilateral Filter", gray)

edged = cv2.Canny(gray, 40, 200)
cv2.imshow("4 - Canny Edges", edged)
# cv2.imwrite("Final_Image.jpg", edged)

cv2.imwrite("Final_Image.jpg", edged)

# Configuration for tesseract

reader = easyocr.Reader(['en'])
ocr_result = reader.readtext(region)
print(ocr_result,'1')

ocr_result = reader.readtext(edged)
print(ocr_result,'edged')

# ocr_result = reader.readtext(gray)
# print(ocr_result,'gray')

# Run tesseract OCR on image
text = pytesseract.image_to_string(region)
print(text)
# #Data is stored in CSV file
# raw_data = {'date': [time.asctime( time.localtime(time.time()) )], 
#         'v_number': [text]}

# df = pd.DataFrame(raw_data, columns = ['date', 'v_number'])
# df.to_csv('data.csv')

# # Print recognized text
# print(text)


CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


[([[23, 7], [169, 7], [169, 45], [23, 45]], 'MH 20 Ee 0249|', 0.05166531459375664)] 1
[([[24, 12], [116, 12], [116, 42], [24, 42]], 'CH 20 CE', 0.05596045791385176)] edged



In [196]:
# !pip install imutils

In [34]:
import os
import cv2 
import easyocr
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

detection_model = tf.saved_model.load('saved_model')
category_index = label_map_util.create_category_index_from_labelmap('label_map.pbtxt')
@tf.function
def detect_fn(image):
    model = detection_model.signatures['serving_default']
    detections = model(image)
    return detections

category_index = label_map_util.create_category_index_from_labelmap('label_map.pbtxt')

def filter_text(region, ocr_result, region_threshold):
    rectangle_size = region.shape[0]*region.shape[1]
    
    plate = [] 
    for result in ocr_result:
        length = np.sum(np.subtract(result[0][1], result[0][0]))
        height = np.sum(np.subtract(result[0][2], result[0][1]))
        
        if length*height / rectangle_size > region_threshold:
            plate.append(result[1])
    return plate

def ocr_it(image, detections, detection_threshold, region_threshold):
    
    # Scores, boxes and classes above threhold
    scores = list(filter(lambda x: x> detection_threshold, detections['detection_scores']))
    boxes = detections['detection_boxes'][:len(scores)]
    classes = detections['detection_classes'][:len(scores)]
    
    # Full image dimensions
    width = image.shape[1]
    height = image.shape[0]
    
    # Apply ROI filtering and OCR
    for idx, box in enumerate(boxes):
        roi = box*[height, width, height, width]
        region = image[int(roi[0]):int(roi[2]),int(roi[1]):int(roi[3])]
        reader = easyocr.Reader(['en'])
        ocr_result = reader.readtext(region)
        length = len(ocr_result)
#         print(ocr_result,'ocr')
        text = filter_text(region, ocr_result, region_threshold)
        
        plt.imshow(cv2.cvtColor(region, cv2.COLOR_BGR2RGB))
        plt.show()
#         print(text)
        return text, region,length

# def process_image(iamge_name, lang_code):
#     return pytesseract.image_to_string(Image.open(iamge_name), lang=lang_code)

def predict(image,output_directory):
    img = cv2.imread(image)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    image_np = np.array(img)
    label_id_offset = 0
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0))
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
    detections['num_detections'] = num_detections

    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
    image_np_with_detections = image_np.copy()

    counter = viz_utils.visualize_boxes_and_labels_on_image_array(
    image_np_with_detections,
    detections['detection_boxes'],
    (detections['detection_classes']+label_id_offset).astype(int),
    detections['detection_scores'],
    category_index,
    use_normalized_coordinates=True,
    #max_boxes_to_draw=1,
    min_score_thresh=.3,
    line_thickness=8,

    agnostic_mode=False
    )
    cv2.imwrite('image_pred.jpg',image_np_with_detections)
#     print(detections)
#     # pytessercat
#     pytesseract.pytesseract.tesseract_cmd = "C:\\Program Files\\Tesseract-OCR\\tesseract"
    text, region,length = ocr_it(image_np_with_detections, detections, 0.3, 0.5)
    if len(text)>0:
        return text[0]
    else:
        image=region
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        #cv2.imshow("1 - Grayscale Conversion", gray)

        gray = cv2.bilateralFilter(gray, 11, 17, 17)
        cv2.imshow("2 - Bilateral Filter", gray)

        edged = cv2.Canny(gray, 40, 200)
        reader = easyocr.Reader(['en'])
        cv2.imwrite('image.jpg',gray)
        ocr_result = reader.readtext(gray)
        print(ocr_result)
        if len(ocr_result)==0:
            return "Could not detect, Try placing another image."
        else:
            ans = ""
            for i in range(len(ocr_result)):
                ans += ocr_result[i][1]
                ans += " "
        return ans

    
    # config = ('-l eng --oem 1 --psm 3')

    # # Run tesseract OCR on image
    # text = pytesseract.image_to_string(image_np_with_detections, config=config)
    # print(text)
    # text = pytesseract.image_to_string(image_np_with_detections,lang='eng')
#     output_file = "image.jpg"
#     cv2.imwrite(output_file,image_np_with_detections)   
predict('data/test/Cars418.png','')


CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
c:\users\shailesh gadde\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:55: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


[([[129.2103500411843, 21.273120131789824], [212.40377743877107, 7.924862668599892], [218.7896499588157, 64.72687986821018], [136.59622256122893, 78.07513733140011]], 'AND', 0.994222343064215), ([[10.645645221012906, 44.40780661883614], [130.7895307737929, 26.065367435286376], [137.3543547789871, 85.59219338116387], [16.210469226207096, 103.93463256471362]], 'OX6s', 0.49288660287857056)]


'AND OX6s '

In [ ]:

img_rgb = cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB)
predicted_result = pytesseract.image_to_string(image_np_with_detections, lang ='eng',
    config ='--oem 3 --psm 6 -c tessedit_char_whitelist = ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')

# Now by using Tessaract OCR

In [ ]:
!pip install pytesseract

In [37]:
from PIL import Image
import pytesseract

import cv2

In [ ]:
!where pytesseract

In [ ]:
def process_image(iamge_name, lang_code):
	return pytesseract.image_to_string(Image.open(iamge_name), lang=lang_code)

def print_data(data):
	print(data)

def output_file(filename, data):
	file = open(filename, "w+")
	file.write(data)
	file.close()

def main():
	data_eng = process_image("test_eng.png", "eng")
	data_ben = process_image("test_ben.png", "ben")
	print_data(data_eng)
	print_data(data_ben)
	output_file("eng.txt", data_eng)
	output_file("ben.txt", data_ben)

if  __name__ == '__main__':
	main()